In [1]:
import os
from time import strftime, localtime
import numpy as np

class _mol2_:

    def _mol2_to_pcd(self, ifile_, color_):
        """ Extracts coordinates from mol2 files and convert into pcd format """

        if ifile_[-5:] != '.mol2':
            print("incorrect file extension")
            print("file format may be wrong --> no output")

        try:
            with open(ifile_, "r") as f:
                mol2 = f.read()

            # extract atoms
            atom_area = mol2.split('@<TRIPOS>ATOM\n')[1].split('@<TRIPOS>')[0]
            atoms_tmp = atom_area.split('\n')
            # print(atoms)
            del atoms_tmp[-1]
            atoms = []
            for atm in atoms_tmp:
                if atm == '':
                    continue
                atoms.append(atm)

        except:
            print("Cannot process mol2 {}".format(ifile_))
            return -1, None, None


        ofilename = os.path.basename(ifile_).replace("mol2", "pcd")    
        try:
            ofile = open(ofilename, "w")
        except IOError:
            print("mol2_to_pcd: Cannot write to current directory: "
                  "{}. Please check for access rights.".format(os.getcwd()))
            ofile.close()
            return -1, None, None

        properties = []
        colors = []
        ofile.write("VERSION .7\nFIELDS x y z rgb\nSIZE 4 4 4 4\n"
                    "TYPE F F F F\nCOUNT 1 1 1 1\n")
        ofile.write("WIDTH {0}\nHEIGHT 1\nVIEWPOINT 0 0 0 1 0 0 0\n"
                    "POINTS {0}\nDATA ascii".format(len(atoms)))
        #print(atoms)
        for atm in atoms:
            cols = atm.split()
            index = int(cols[0])
            atom = str(cols[1])
            x = float(cols[2])
            y = float(cols[3])
            z = float(cols[4])
            properties.append([index, atom])
            colors.append(color_[atom])
            ofile.write("\n{} {} {} {}".format(x, y, z, color_[atom]))

        ofile.close()                                                    
        
        #print(ofilename)
        return ofilename, properties, colors



class _pcd_:

    def _get_coordinates(self, ifile_):

        if ifile_[-4:] != '.pcd':
            print("incorrect file extension")
            print("file format may be wrong --> no output")
            if "DATA ascii" not in open(ifile_, 'r').read():
                return -1

        coordinates = []
        errors = []
        with open(ifile_, 'r') as f:
            data = f.read().split('\n')
        data = [l for l in data[10:] if l != '']

        for line_ in data:
            x, y, z, rgb = line_.split()
            try: 
                x = float(x)
            except ValueError:
                errors.append(-1)
            try:
                y = float(y)
            except ValueError:
                errors.append(-1)
            try:
                z = float(z)
            except ValueError:
                errors.append(-1)
            try:
                rgb = int(rgb)
            except ValueError:
                errors.append(-1)

            if -1 in errors:
                print("Errors while parsing PCD")
                break

            coordinates.append([x, y, z, rgb])

        if -1 not in errors:
            self.coordinates = coordinates
            return coordinates
        else:
            return -1



    def _write_mol2(self, ofile_, coordinates_, atom_, atom_type_, residue_, 
                                                    macromol_="PROTEIN"):

        
        if ofile_[-5:] != '.mol2':
            ofile_ += '.mol2'
        name = os.path.basename(ofile_)
        name = os.path.splitext(name)[0]

        of_string = ""
        of_string += "# Modified by ProCare\n"
        of_string += "# Modification time: {}\n".format(
                                strftime("%a %d %b %Y %H:%M:%S", localtime()))
        of_string += "# Name: {}.mol2\n\n".format(name)

        of_string += "@<TRIPOS>MOLECULE\n"
        of_string += "{}\n".format(name)
        of_string += "{:>5}{:>6}{:>6}{:>6}{:>6}\n".format(
                                                len(coordinates_), 0, 0, 0, 0)
        of_string += "{}\n".format(macromol_)
        of_string += "NO_CHARGES\n"
        of_string += "@<TRIPOS>ATOM"

        for i, point in enumerate(coordinates_):
            x, y, z, rgb = [*point]
            of_string += ("\n{:>7} {:<8} {:>9.4f} {:>9.4f} {:>9.4f} "
                          "{:<5} {:>5} {:<8} {:>9}".format(i+1,
                                                     atom_[rgb],
                                                     x,
                                                     y,
                                                     z,
                                                     atom_type_[rgb],
                                                     i+1,
                                                     residue_[rgb]+str(i+1),
                                                     0.0000
                                                    ))
        of_string += "\n@<TRIPOS>BOND"
        
        with open(ofile_, 'w') as of:
            of.write(of_string)
        print("written mol2 to {}".format(ofile_))
        self.type = "pcd"
        return ofile_

    
    def _pcd_to_mol2(self, ifile_, atom_, atom_type_, residue_, 
                                                    macromol_="PROTEIN"):
                
        coordinates = self._get_coordinates(ifile_)
        if coordinates != -1:
            ofile = os.path.basename(ifile_).replace('pcd', 'mol2')
            if self._write_mol2(ofile, coordinates, atom_, atom_type_, 
                                                residue_, macromol_) == ofile:
                self.ifile = ifile_
                return ofile
            else:
                return -1


class _volsite_cavity_(_mol2_, _pcd_):
    
    def __init__(self):

        __COLOR = {"OG":8204959,
                   "N":30894,
                   "O":15219528,
                   "NZ":15231913,
                   "CZ":4646984,
                   "CA":16741671,
                   "DU":7566712,
                   "OD1":0,}


        __ATOM = {val:key for key, val in __COLOR.items()}


        __ATOM_TYPE = {"OG":"O.3",
                          "N":"N.am",
                          "O":"O.2",
                          "NZ":"N.4",
                          "CZ":"C.ar",
                          "CA":"C.3",
                          "DU":"H",
                          "OD1":"O.co2",}

        __RESIDUE = {"OG":"SER",
                        "N":"ALA",
                        "O":"ALA",
                        "NZ":"LYS",
                        "CZ":"PHE",
                        "CA":"GLY",
                        "DU":"CUB",
                        "OD1":"ASP",}

        self.COLOR = __COLOR

        self.ATOM = __ATOM

        self.ATOM_TYPE = {key:__ATOM_TYPE[val] 
                                for key, val in __ATOM.items()}
        self.RESIDUE = {key:__RESIDUE[val] 
                                for key, val in __ATOM.items()}

        

    def mol2_to_pcd(self, ifile_):
        return self._mol2_to_pcd(ifile_, self.COLOR)
        



    def pcd_to_mol2(self, ifile_):
        return self._pcd_to_mol2(ifile_, self.ATOM,
                                         self.ATOM_TYPE,
                                         self.RESIDUE)



    def write_mol2(self, ofile_, coordinates_):
        return self._write_mol2(ofile_, coordinates_, self.ATOM, self.ATOM_TYPE,
                    self.RESIDUE)

In [2]:
source_cavity = _volsite_cavity_()
source_file, source_prop, source_color = source_cavity.mol2_to_pcd("cfpfh_2rh1_cavity.mol2")
print("##############################source_file##############################")
print(source_file)
print("##############################source_prop##############################")
print(source_prop[0:10])
print("##############################source_color##############################")
print(source_color[0:10])

##############################source_file##############################
cfpfh_2rh1_cavity.pcd
##############################source_prop##############################
[[1, 'OG'], [2, 'OG'], [3, 'OG'], [4, 'OG'], [5, 'OG'], [6, 'OG'], [7, 'OG'], [8, 'OG'], [9, 'OG'], [10, 'OG']]
##############################source_color##############################
[8204959, 8204959, 8204959, 8204959, 8204959, 8204959, 8204959, 8204959, 8204959, 8204959]


In [3]:
print(source_prop[0][1])

OG


In [13]:
source = read_point_cloud(source_file)

ModuleNotFoundError: No module named 'procare'

In [15]:
# test .pcd rgb

rgb = 30894
r = (rgb>>16)&0x0000ff
g = (rgb>>8)&0x0000ff
b = (rgb)&0x0000ff

print(r)
print(g)
print(b)

# red
r = 255
g = 0
b = 0
rgb = (r << 16 | g << 8 | b)
print("red: ", rgb)

# green
r = 0
g = 255
b = 0
rgb = (r << 16 | g << 8 | b)
print("green: ", rgb)

# blue
r = 0
g = 0
b = 255
rgb = (r << 16 | g << 8 | b)
print("blue: ", rgb)

# yellow
r = 255
g = 255
b = 0
rgb = (r << 16 | g << 8 | b)
print("yellow: ", rgb)

0
120
174
red:  16711680
green:  65280
blue:  255
yellow:  16776960
